In [22]:
import numpy as np
import bz2
import pandas as pd

# Load sequences from compressed BZ2 file
def load_sequences(filename):
    sequences = {} 
    try:
        with bz2.open(filename, 'rt') as file: 
            for line in file:
                parts = line.strip().split(' \\ ')
                if len(parts) == 2:
                    # Assign the parts to gene_id and sequence variables
                    gene_id, sequence = parts[0].strip(), parts[1].strip()
                    # Map gene ID to its sequence in the dictionary
                    sequences[gene_id] = sequence
                else:
                    print(f"Skipping malformed line: {line.strip()}")
    except Exception as e:
        print(f"Error reading file: {e}")
    print(f"Total sequences loaded: {len(sequences)}")
    return sequences

# Calculate PWM using counts matrix and pseudocounts
def compute_pwm(counts):
    pseudocount = 1  # pseudocount is to prevent division by zero
    # Add pseudocount to each count in the matrix
    counts_pseudocount = counts + pseudocount
    # Calculate the sum of counts for each column
    total_counts = counts_pseudocount.sum(axis=0)
    # Divide counts by total to get frequencies and convert to PWM
    frequency_matrix = counts_pseudocount / total_counts
    background_frequency = 0.25  # Set a uniform background frequency
    # Calculate the log-ratio of frequency matrix to background frequency
    return np.log2(frequency_matrix / background_frequency)

# Compute an adjusted frequency matrix using pseudocounts
def compute_adjusted_frequency(counts):
    pseudocount = 1 
    # Calculate frequency with added pseudocounts
    return (counts + pseudocount) / (counts.sum(axis=0) + 4 * pseudocount)

# top binding sites using the PWM
def scan_sequences(pwm, sequences, top_n=30):
    # Map DNA bases to indices
    base_mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    motif_length = pwm.shape[1]  # Determine the length of the motif
    scores = []  # List to hold scores for each potential binding site

    # Iterate through all sequences to find binding sites
    for gene_id, seq in sequences.items():
        if len(seq) < motif_length:
            # Skip sequences that are too short to contain the motif
            print(f"Skipping {gene_id} due to short sequence length {len(seq)}")
            continue
        for i in range(len(seq) - motif_length + 1):
            subseq = seq[i:i+motif_length] 
            # Calculate the score based on the PWM
            score = sum(pwm[base_mapping.get(b, 0), j] for j, b in enumerate(subseq) if b in base_mapping)
            # Append the score and subsequence information to the list
            scores.append((gene_id, score, subseq, i))

    # Sort the list of scores in descending order and get the top N
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_n]

# File path for the bzipped sequence data
bzipped_path = 'E_coli_K12_MG1655.400_50.bz2'
# Load the sequences from the given path
sequences = load_sequences(bzipped_path)
if sequences:
    print("Sequences loaded successfully.")
else:
    print("No sequences were loaded.")

# Load the count matrix using pandas
rows = []
with open('argR-counts-matrix.txt', 'r', encoding='utf-8-sig') as f:
    for line in f:
        if line.strip():
            parts = line.strip().split('\t|\t')
            base = parts[0].strip().lower()
            counts = list(map(int, parts[1].strip().split('\t')))
            rows.append([base] + counts)

# Build the DataFrame
df_counts = pd.DataFrame(rows)
df_counts.set_index(0, inplace=True)

# Convert DataFrame to a NumPy array of floats
counts = df_counts.astype(float).to_numpy()

# Generate PWM and the adjusted frequency matrix
pwm = compute_pwm(counts)
adjusted_frequency = compute_adjusted_frequency(counts)

# Display the calculated matrices
print("PWM (Position Weight Matrix):")
print(pwm)
print("\nAdjusted Frequency Matrix F'(b, j) with Pseudocounts:")
print(adjusted_frequency)

# Identify the top binding sites in the sequences using the PWM
top_binding_sites = scan_sequences(pwm, sequences)

# Print out the top 30 binding sites found
if top_binding_sites:
    print("Top 30 binding sites:")
    for site in top_binding_sites:
        # Show details of each binding site
        print(f"Gene ID: {site[0]}, Score: {site[1]:.2f}, Position: {site[3]}, Subseq: {site[2]}")
else:
    print("No significant binding sites detected.")


Total sequences loaded: 4319
Sequences loaded successfully.
PWM (Position Weight Matrix):
[[ 0.21572869  0.74624341  1.50523531  0.36773178 -0.63226822 -1.36923381
   1.50523531  1.50523531 -0.95419631  0.50523531  0.21572869 -0.36923381
   0.04580369  1.74624341 -0.63226822 -1.36923381 -1.36923381  1.74624341]
 [ 0.04580369 -0.63226822 -1.95419631 -0.14684139 -1.36923381 -0.95419631
  -0.95419631 -0.95419631 -1.95419631 -2.95419631 -1.36923381 -2.95419631
   0.04580369 -2.95419631 -0.95419631 -0.95419631  1.68965988 -2.95419631]
 [-0.95419631 -1.36923381 -1.95419631  0.21572869 -1.36923381  1.50523531
  -1.36923381 -1.36923381 -2.95419631 -1.95419631 -2.95419631 -1.95419631
  -2.95419631 -1.95419631 -2.95419631  1.04580369 -2.95419631 -1.36923381]
 [ 0.36773178  0.36773178 -0.63226822 -0.63226822  1.36773178 -1.95419631
  -1.95419631 -1.95419631  1.63076619  1.13326653  1.21572869  1.50523531
   0.85315861 -1.95419631  1.43812111  0.04580369 -1.95419631 -2.95419631]]

Adjusted Frequen